## 1. Model

In [2]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split

import pandas_profiling

In [2]:
compare_clf = LogisticRegression(C=10, solver='lbfgs', max_iter = 1000)

## 2. Preparing your dataset for modeling

In [3]:
cols = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation',
          'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'y']

In [4]:
adult_train = pd.read_csv('Census/adult.data', header = None)
adult_test  = pd.read_csv('Census/adult.test', header = None)
adult_train.columns = cols
adult_test.columns = cols

In [5]:
# remove whitespaces from the beginning of categorical values
for col in adult_train.columns:
    if col not in ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']:
        adult_test[col] = adult_test[col].str.strip()
        adult_train[col] = adult_train[col].str.strip()

adult_train.head(5)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,y
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
adult_test.head(5)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,y
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


## 3. Strategy 1: Do not deal with missing values.

In [7]:
from copy import deepcopy
test, train = deepcopy(adult_test), deepcopy(adult_train)

# 3.1 drop rows with missing values
test, train = test.replace('?', pd.np.nan).dropna(), train.replace('?', pd.np.nan).dropna()

# 3.2 categorical to numerical
cols_to_transform = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']
test, train = pd.get_dummies(test, columns=cols_to_transform), pd.get_dummies(train, columns=cols_to_transform)

# transform y
train['y']= train.y.eq('>50K').mul(1)
test['y']= test.y.eq('>50K.').mul(1)

# add missing column to test dataset
test['native_country_Holand-Netherlands'] = 0

In [8]:
x_train, y_train = train.loc[:, train.columns != 'y'], train['y']
x_test, y_test = test.loc[:, test.columns != 'y'], test['y']
# 3.3
compare_clf.fit(x_train, y_train)
drop_predicted = compare_clf.predict(x_test)
# 3.4
drop_train_score = compare_clf.score(x_train, y_train)
drop_test_score = accuracy_score(y_test, drop_predicted)
drop_cv_scores = cross_val_score(LogisticRegression(C=10, solver='lbfgs', max_iter = 1000), x_train.append(x_test, sort=False), y_train.append(y_test), scoring='accuracy', cv=20)
drop_cv_scores.mean()

0.7927332617316755

###### 3.5
The obvious and probably the most bad thing is loosing data, therefore some info when deleteting missing values. For example sometimes those NAs are there for a reason, and they can represent some patterns, that can be important in data analysis. So when we delete the values we just miss an opportunity to observe such patterns. and of course other valuable onfo

## 4. Strategy 2: Global most common substitution.

In [9]:

# 4.1 Global most common substitution
test, train = deepcopy(adult_test), deepcopy(adult_train)

numeric_imputer = SimpleImputer(missing_values = 0, strategy = 'median')
test[['capital_loss', 'capital_gain']] = numeric_imputer.fit_transform(test[['capital_loss', 'capital_gain']])
train[['capital_loss', 'capital_gain']] = numeric_imputer.fit_transform(train[['capital_loss', 'capital_gain']])

categoric_imputer = SimpleImputer(missing_values= '?',strategy='most_frequent')
train[['workclass', 'occupation', 'native_country']] = categoric_imputer.fit_transform(train[['workclass', 'occupation', 'native_country']])
test[['workclass', 'occupation', 'native_country']] = categoric_imputer.fit_transform(test[['workclass', 'occupation', 'native_country']])

# 4.2 categorical to numerical
cols_to_transform = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']
test, train = pd.get_dummies(test, columns=cols_to_transform), pd.get_dummies(train, columns=cols_to_transform)

# transform y
train['y']= train.y.eq('>50K').mul(1)
test['y']= test.y.eq('>50K.').mul(1)

# add missing column to test dataset
test['native_country_Holand-Netherlands'] = 0

In [10]:
x_train, y_train = train.loc[:, train.columns != 'y'], train['y']
x_test, y_test = test.loc[:, test.columns != 'y'], test['y']
# 4.3
compare_clf.fit(x_train, y_train)
comm_predicted = compare_clf.predict(x_test)
# 4.4
comm_train_score = compare_clf.score(x_train, y_train)
comm_test_score = accuracy_score(y_test, comm_predicted)
comm_cv_scores = cross_val_score(LogisticRegression(C=10, solver='lbfgs', max_iter = 1000), x_train.append(x_test, sort=False), y_train.append(y_test), scoring='accuracy', cv=20)
comm_cv_scores.mean()

0.7799837335414785

###### 4.5
Useful when values are missing at random so that the mean of missing values is similar to mean of the whole column. When the situation is different (missing values have different mean), this kind of imputation may cause a bias the data.

## 5. Strategy 3: Regression imputation.
5.1. For each categorical column with missing values, create a dataset in which this column is a target
variable. In the dataset obtained, fill the other missing values using methods described in Strategy 2.

5.2. Train logistic regression model for each of those datasets (5.1.). Use this model to predict missing values.

5.3. For each numerical column with missing values, create a dataset in which this column is a target
variable. In the dataset obtained, fill the other missing values using mathods described in Strategy 2.

5.4. Train linear regression model for each for those datasets (5.3.). Use this model to predict missing values.

5.5. After filling all the missing values using regression imputation, repeat steps 3.3. - 3.4. with your dataset.

5.6. What are the main drawbacks of this approach? Describe in 3-4 sentences.

In [12]:
ds = adult_train.append(adult_test, sort=False)

ds['native_country'].unique()

array(['United-States', 'Cuba', 'Jamaica', 'India', '?', 'Mexico',
       'South', 'Puerto-Rico', 'Honduras', 'England', 'Canada', 'Germany',
       'Iran', 'Philippines', 'Italy', 'Poland', 'Columbia', 'Cambodia',
       'Thailand', 'Ecuador', 'Laos', 'Taiwan', 'Haiti', 'Portugal',
       'Dominican-Republic', 'El-Salvador', 'France', 'Guatemala',
       'China', 'Japan', 'Yugoslavia', 'Peru',
       'Outlying-US(Guam-USVI-etc)', 'Scotland', 'Trinadad&Tobago',
       'Greece', 'Nicaragua', 'Vietnam', 'Hong', 'Ireland', 'Hungary',
       'Holand-Netherlands'], dtype=object)

In [13]:
Namerica = set(['United-States', 'Canada'])
Camerica = set(['Mexico', 'Honduras', 'El-Salvador', 'Guatemala', 'Nicaragua'])
asia = set(['India' ,'Iran' ,'Cambodia','Thailand' ,'Laos' ,'Taiwan','China' ,'Japan' ,'Vietnam','Hong'])
oceania =  set(['Philippines'])
europe =  set(['England','Germany','Italy','Poland','Portugal','France','Yugoslavia','Scotland','Greece','Ireland','Hungary','Holand-Netherlands'])
Samerica =  set(['South','Columbia','Ecuador','Peru'])
caribbean = set(['Dominican-Republic','Haiti','Trinadad&Tobago','Cuba','Jamaica','Puerto-Rico','Outlying-US(Guam-USVI-etc)'])

ds.loc[[(x in europe) for x in ds['native_country']] , 'native_country'] = 'Europe'
ds.loc[[(x in Camerica) for x in ds['native_country']] , 'native_country'] = 'Central-America'
ds.loc[[(x in asia) for x in ds['native_country']], 'native_country'] = 'Asia'
ds.loc[[(x in oceania) for x in ds['native_country']], 'native_country'] = 'Oceania'
ds.loc[[(x in Namerica) for x in ds['native_country']], 'native_country'] = 'North-America'
ds.loc[[(x in Samerica) for x in ds['native_country']], 'native_country'] = 'South-America'
ds.loc[[(x in caribbean) for x in ds['native_country']], 'native_country'] = 'Caribbean'

In [14]:
ds['native_country'].unique()

array(['North-America', 'Caribbean', 'Asia', '?', 'Central-America',
       'South-America', 'Europe', 'Oceania'], dtype=object)

In [16]:
# 5.1-2
# 5.1. For each categorical column with missing values, create a dataset in which this column is a target variable.
# In the dataset obtained, fill the other missing values using methods described in Strategy 2.

# 5.2. Train logistic regression model for each of those datasets (5.1.). Use this model to predict missing values.

mis_val_cat = ['workclass', 'occupation', 'native_country']
cat = ['workclass','education', 'marital_status', 'occupation','relationship', 'race', 'sex', 'native_country', 'y']
print([column for column in mis_val_cat if column != col])
for col in mis_val_cat:
       
    numeric_imputer = SimpleImputer(missing_values = 0, strategy = 'median')
    test[['capital_loss', 'capital_gain']] = numeric_imputer.fit_transform(test[['capital_loss', 'capital_gain']])
    train[['capital_loss', 'capital_gain']] = numeric_imputer.fit_transform(train[['capital_loss', 'capital_gain']])

    categoric_imputer = SimpleImputer(missing_values='?', strategy='most_frequent')
    train[[column for column in mis_val_cat if column != col]] = categoric_imputer.fit_transform(train[[column for column in mis_val_cat if column != col]])
    test[[column for column in mis_val_cat if column != col]] = categoric_imputer.fit_transform(test[[column for column in mis_val_cat if column != col]])
    
    adults_df = adult_train.append(adult_test, sort=False)
    adults_df = pd.get_dummies(adults, columns= [column for column in cat if column != col])
    
    train = adults[adults[col] != 0]
    unknown = adults[adults[col] == 0]

    y_train = train[col]
    x_train = train.loc[:, train.columns != col]
    y_unknown = unknown[col]
    x_unknown = unknown.loc[:, unknown.columns != col]
    
    model = LogisticRegression()
    model.fit(x_train, y_train)
    
    y_unknown = model.predict(x_unknown)
    unknown[col] = y_unknown
    
    adults_df = train.append(unknown, ignore_index=True)
    
    ds[col] =  adults_df[col]

['occupation', 'native_country']


KeyError: "['occupation' 'native_country'] not in index"

In [1]:
# 5.3-4
# 5.1. For each categorical column with missing values, create a dataset in which this column is a target variable.
# In the dataset obtained, fill the other missing values using methods described in Strategy 2.

# 5.2. Train logistic regression model for each of those datasets (5.1.). Use this model to predict missing values.

mis_val_num = ['capital_loss', 'capital_gain']

for col in mis_val_cat:
       
    numeric_imputer = SimpleImputer(missing_values = 0, strategy = 'median')
    test[[column for column in mis_val_num if column != col]] = numeric_imputer.fit_transform(test[[column for column in mis_val_cat if column != col]])
    train[[column for column in mis_val_num if column != col]] = numeric_imputer.fit_transform(train[[column for column in mis_val_cat if column != col]])

    categoric_imputer = SimpleImputer(missing_values= '?',strategy='most_frequent')
    train[['workclass', 'occupation', 'native_country']] = categoric_imputer.fit_transform(train[['workclass', 'occupation', 'native_country']])
    test[['workclass', 'occupation', 'native_country']] = categoric_imputer.fit_transform(test[['workclass', 'occupation', 'native_country']])

    adults = adult_train.append(adult_test, sort=False)
    adults = pd.get_dummies(adults, columns= [column for column in cat if column != col])
    
    train = adults[adults[col] != 0]
    unknown = adults[adults[col] == 0]

    y_train = train[col]
    x_train = train.loc[:, train.columns != col]
    y_unknown = unknown[col]
    x_unknown = unknown.loc[:, unknown.columns != col]
    
    model = LogisticRegression()
    model.fit(x_train, y_train)
    
    y_unknown = model.predict(x_unknown)
    unknown[col] = y_unknown
    
    adults_df = train.append(unknown, ignore_index=True)
    
    ds[col] =  adults_df[col]

IndentationError: unexpected indent (<ipython-input-1-6d31bd4f74ba>, line 24)

In [11]:
df2 = pd.DataFrame(np.random.randint(low=0, high=10, size=(5, 5)), columns=['a', 'b', 'c', 'd', 'e'])
df2

,a,b,c,d,e
0,0,3,2,9,4
1,3,6,1,0,2
2,1,3,7,7,2
3,7,5,3,6,7
4,7,6,9,8,4


## 6. Conclusions
6.1. What strategy of dealing with missing values gave you the best classification accuracy? (3-4 sentences)
6.2. Describe what are the use-cases for different methods of missing value imputation? (4-5 sentences)

## 7. Strategy 4: KNN imputation.
7.1. Impute missing values in Census dataset using KNN algorithm. You can use library utilities for this task
as well.
7.2. - 7.4. Same as 3.2. - 3.4.
7.5. Compare the results with those obtained using Strategies 1-3.

In [3]:
df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
                   'B': [5, 6, 7, 8, 9],
                   'C': ['a', 'b', 'c', 'd', 'e']})

In [4]:
df.replace({'A': {0: 100, 4: 400}})

,A,B,C
0,100,5,a
1,1,6,b
2,2,7,c
3,3,8,d
4,400,9,e


In [5]:
df

,A,B,C
0,0,5,a
1,1,6,b
2,2,7,c
3,3,8,d
4,4,9,e
